In [1]:
%pylab inline
import tensorflow as tf
#from src.setup.config import db
from itertools import groupby
import pandas as pd
from collections import OrderedDict
from datetime import date
import pyodbc

Populating the interactive namespace from numpy and matplotlib


C:\Users\Vivi\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Data Preprocessing (can be skipped after done once)

In [2]:
records=pd.read_csv('C:/Users/Vivi/Desktop/hack4health_RKI-data-scripts/survstat/csv/cam_5y.csv',encoding='latin-1')
CoNew = pd.read_csv('C:/Users/Vivi/Desktop/hack4health_RKI-data-scripts/DieInfluenza/CoordBins.csv',encoding='latin-1')

In [5]:
recordsX=records[['county','week','count','incidence']]
gb = recordsX.groupby(['week','county']).sum()

In [ ]:
gb.index[0]

In [6]:
newDF = pd.DataFrame(columns=['week','year','county','count','incidence','X','Y','BinX','BinY','LocCounty'])
x = 0
for i in gb.index:
    week = i[0]
    counties = i[1]
    counts = gb.values[x][0]
    incidences = gb.values[x][1]
    loc = CoNew[CoNew['county']==counties]
    if loc['X'].values>0:
        newDF=newDF.append({'week': str(week)[7:],'year':str(week)[:4],'county':str(counties),'count':str(counts),'incidence':str(incidences),'X':loc['X'].values,'Y':loc['Y'].values,'BinX':loc['BinX'].values,'BinY':loc['BinY'].values,'LocCounty':loc['county'].values}, ignore_index=True)    
    try:
        with open('StomachVirusInput.csv', 'a') as f:
            newDF.to_csv(f, header=False)
    except:
        pass
    newDF=pd.DataFrame(columns=['week','year','county','count','incidence','X','Y','BinX','BinY','LocCounty'])
    x = x+1
#newDF.to_csv('StomachVirusInput.csv')

C:\Users\Vivi\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if __name__ == '__main__':


In [2]:
newDF = pd.read_csv('StomachVirusInput.csv',encoding='latin-1')
newDF.columns = ['ID','week','year','county','count','incidence','X','Y','BinX','BinY','LocCounty']#.set_index('Unnamed: 0')
newDF = newDF.dropna()
newDF = newDF[['week','year','county','count','incidence','X','Y','BinX','BinY']].reset_index(drop=True)
newDF.to_csv('StomachVirusNew.csv')

In [ ]:
newDF = pd.read_csv('StomachVirusInput.csv',encoding='latin-1')
newDF

In [5]:
test = newDF[newDF['year']==2001][newDF[newDF['year']==2001]['week']==2]['incidence']

In [6]:
def FindLabels(data):
    LabelDF = pd.DataFrame(columns=['count','incidence','county'])
    for w in data.index:
        try:
            thisyear = data.iloc[w]['year']
            thisweek = data.iloc[w]['week']
            thisarea = data.iloc[w]['county']
            if int(thisweek)<52:
                nextweek = int(thisweek)+1
                nextyear = int(thisyear)
            else:
                nextweek = 1
                nextyear = int(thisyear)+1
            nextdf = data[data['year']==nextyear][data[data['year']==nextyear]['week']==nextweek]
            nextcount = np.sum(nextdf[nextdf['county']==thisarea]['count'])
            if nextcount>0:
                nextincidence = np.sum(nextdf[nextdf['county']==thisarea]['incidence'])
                nextcounty = nextdf[nextdf['county']==thisarea]['county'].values[0]
            else:
                nextincidence=0  
                nextcounty = 'no Data'
        except:
            nextcount=0
            nextincidence=0  
            nextcounty = 'no Data'
        LabelDF=LabelDF.append({'count':nextcount,'incidence':nextincidence,'county':nextcounty}, ignore_index=True)
        if (w % 1000 == 0):
            print(w)
    return LabelDF
        
label = FindLabels(newDF)
label.to_csv('StomachOutput.csv')

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


# Load Processed Data

In [84]:
Labels=pd.read_csv('StomachOutput.csv',encoding='latin-1').set_index('Unnamed: 0')
Data=pd.read_csv('StomachVirusInput.csv',encoding='latin-1').set_index('Unnamed: 0')
Data = Data[['week','year','count','incidence','X','Y']]
dfboth = Data.join(Labels,lsuffix='x',how='outer').dropna()
Data = dfboth[['week','year','countx','incidencex','X','Y']].rename(index=str, columns={"countx": "count", "incidencex": "incidence"})
Labels = dfboth[['count','incidence']]

In [86]:
Data

week  year  count   incidence              X              Y
Unnamed: 0                                                             
0              1  2001    1.0   19.892800    [7.7270285]  [50.73946897]
1              1  2001    1.0   34.366229   [8.03291064]  [53.22147688]
2              1  2001    1.0   18.432613  [12.17414403]  [51.77891857]
3              1  2001    5.0  101.440870    [8.0989165]  [49.43943465]
4              1  2001    1.0   24.270536  [13.72397933]  [52.82678413]
5              1  2001    2.0   21.857354  [14.20996868]   [51.3147493]
6              1  2001    1.0   22.757731   [6.94707509]   [49.9288013]
7              1  2001    2.0   37.901348    [9.7384129]  [48.12156599]
8              1  2001    1.0   39.433904   [7.31217013]   [49.7137353]
9              1  2001    1.0   24.896610   [6.41060344]  [50.07655585]
10             1  2001    1.0   13.682698   [7.92684336]  [47.93201209]
11             1  2001    1.0    8.041350    [8.9424957]  [48.67913217]
12             1  2001    2.0   39.132082  [11.42489293]  [52.23817091]
13             1  2001    2.0   36.875060   [7.40450793]  [51.85010267]
14             1  2001    2.0   24.692713  [13.79828137]  [52.06891858]
15             1  2001   12.0  138.037442    [8.7571124]  [49.85856737]
16             1  2001    3.0   80.267329   [9.17704026]  [54.33656702]
17             1  2001    1.0   29.507713  [11.91814199]  [48.07381557]
18             1  2001    2.0   24.702091   [7.34112324]  [51.34042391]
19             1  2001    1.0   20.740433   [8.72869426]  [48.90092033]
20             1  2001    4.0   43.071658  [12.96347656]  [50.62634756]
21             1  2001    4.0   26.680107   [9.36781325]  [48.64586295]
22             1  2001    1.0   35.925992   [8.48869121]  [48.47057301]
23             1  2001    1.0   32.034740   [8.09221282]  [53.63966428]
24             1  2001    2.0   33.676298   [9.75401336]  [50.59158168]
25             1  2001    4.0   66.388017  [11.19630705]  [48.18805341]
26             1  2001    1.0   30.486256   [10.4187703]  [51.85454134]
27             1  2001    2.0   54.967706  [10.69394067]  [50.90878869]
28             1  2001    1.0   32.964014   [7.01640035]  [52.49645878]
29             1  2001    5.0   66.413518  [14.71406248]  [51.24456627]
...          ...   ...    ...         ...            ...            ...
263830        52  2016    3.0   87.894363   [7.58832382]  [50.34567982]
263831        52  2016    4.0   16.463851   [6.95755812]  [50.95978658]
263832        52  2016    7.0   46.823299  [12.37970108]  [51.33833756]
263833        52  2016    2.0   42.259045   [7.01619618]  [51.05635665]
263834        52  2016    2.0   37.178241   [8.39508502]  [49.47786747]
263835        52  2016    2.0   27.961804  [10.74087666]  [53.87789972]
263836        52  2016    2.0   23.063477  [11.66530778]  [52.12460885]
263837        52  2016    3.0   39.661785   [8.23840117]  [49.96545159]
263838        52  2016    5.0   62.720530    [8.5187684]  [49.49029778]
263839        52  2016   11.0   30.684460  [11.55480833]  [48.15608074]
263840        52  2016    5.0   60.916435   [7.62755318]  [51.95097008]
263841        52  2016    1.0   69.479291   [9.99119632]   [54.0816578]
263842        52  2016    5.0   35.173457  [11.26434146]  [49.44199411]
263843        52  2016    5.0   73.724729   [6.85111733]  [51.51384723]
263844        52  2016    1.0   15.771903   [8.22620503]  [53.14845378]
263845        52  2016    1.0  107.674822   [13.4209559]  [48.58370149]
263846        52  2016    1.0   25.439179   [8.71335602]  [48.87577763]
263847        52  2016    1.0   45.619282  [12.10760039]  [47.84382971]
263848        52  2016    2.0   41.706516  [12.12517513]  [54.14248171]
263849        52  2016    1.0   35.687379  [10.36556736]  [52.11665718]
263850        52  2016    1.0   53.506138  [10.22943837]  [50.05813935]
263851        52  2016    5.0   97.242800   [7.07329204]  [51.15819221]
263852        52  2016    4.0   19.680641    [9.1745

## Normalize Data

In [ ]:
import sklearn
from sklearn.preprocessing import normalize
normalized = newDF
"""x = normalized['BinX']
normBinX =sklearn.preprocessing.normalize([x],axis=1)
normalized['BinX']=normBinX[0]
x = normalized['BinY']
normBinY =sklearn.preprocessing.normalize([x])
normalized['BinY']=normBinY[0]
x = normalized['X']
normX =sklearn.preprocessing.normalize([x])
normalized['X']=normX[0]
x = normalized['Y']
normY =sklearn.preprocessing.normalize([x])
normalized['Y']=normY[0]"""
x = normalized['count']
normcount =sklearn.preprocessing.normalize([x])
normalized['count']=normcount[0]
x = normalized['incidence']
normInc =sklearn.preprocessing.normalize([x])
normalized['incidence']=normInc[0]

In [ ]:
normLabels = Labels
x = normLabels['count']
normcount =sklearn.preprocessing.normalize([x])
normLabels['count']=normcount[0]
x = normLabels['incidence']
normInc =sklearn.preprocessing.normalize([x])
normLabels['incidence']=normcount[0]

In [6]:
lenall = Data.shape[0]
trainSize = lenall//3*2#define sizes for data sets (train: 2/3, val:1/6, test:1/6 of whole dataset) 
valSize = lenall//6

#Split data into train, val and test set------------------------------------------
trainingData = Data.iloc[0:trainSize].reset_index(drop=True)
trainingLabels = Labels.iloc[0:trainSize].reset_index(drop=True)
validationData = Data.iloc[trainSize+1:trainSize+valSize].reset_index(drop=True)
validationLabels = Labels.iloc[trainSize+1:trainSize+valSize].reset_index(drop=True)
testData = Data.iloc[trainSize+valSize+1:].reset_index(drop=True)
testLabels = Labels.iloc[trainSize+valSize+1:].reset_index(drop=True)

In [102]:
#Method to get batches------------------------------------------------------------
def getBatches(batchsize,typeBatch):
    if typeBatch=="train":
        numBatches = shape(trainingData)[0]//batchsize
        Batches=[]
        for count in range(numBatches-1):
            batch = trainingData[count*batchsize:(count+1)*batchsize]
            labels = trainingLabels[count*batchsize:(count+1)*batchsize]
            b,l=formatBatch(batch,labels,batchsize)
            Batches.append([b,l])
    elif typeBatch=="val":
        batch = validationData.sample(batchsize)
        labels = validationLabels.iloc[batch.index]
        b,l=formatBatch(batch,labels,batchsize)
        Batches  = [b,l]
    elif typeBatch=="test":
        batch = testData.sample(batchsize)
        labels = testLabels.iloc[batch.index]
        b,l=formatBatch(batch,labels,batchsize)
        Batches  = [b,l]
    else:
        print("typeBatch needs to be set to one of these values: train, val, test.")
        pass
    return Batches
def getLastWeek(thisweek,thisyear):
    weekdata = Data[Data['year']==thisyear][Data[Data['year']==thisyear]['week']==thisweek]
    size = shape(weekdata)[0]
    batch = formatBatch(weekdata,weekdata,size)

    return batch,size
def formatBatch(batch,labels,batchsize):
    batch = batch.reset_index(drop=True)
    labels = labels.reset_index(drop=True)
    newl = []
    newb = []
    for i,b in enumerate(batch['week']):
        x = batch['X'][i][1:-1]
        y = batch['Y'][i][1:-1]
        try:
            newb = np.append(newb,[int(b),int(batch['count'][i])/10,(batch['incidence'][i])/100,float(x)/100,float(y)/100])
        except:
            newb = np.append(newb,[int(b),int(batch['count'][i])/10,(batch['incidence'][i])/100,float(0)/100,float(0)/100])
    for ind,l in enumerate(labels['count']):
        newl = np.append(newl,[l/10,(labels['incidence'][ind])/100],axis=0)
    return np.reshape(newb,(batchsize,5)),np.reshape(newl,(batchsize,2))
        

In [103]:
b,s = getLastWeek(52,2016)
b

(array([[52.        ,  0.1       ,  0.40128411,  0.0700837 ,  0.50463981],
        [52.        ,  0.1       ,  0.33830771,  0.11341941,  0.52705981],
        [52.        ,  0.1       ,  0.27250019,  0.12703571,  0.48216069],
        ...,
        [52.        ,  0.3       ,  0.56352922,  0.09937881,  0.49780485],
        [52.        ,  0.1       ,  1.52749491,  0.07369184,  0.49252331],
        [52.        ,  0.4       ,  0.28640403,  0.06417591,  0.50610519]]),
 array([[0.1       , 0.40128411],
        [0.1       , 0.33830771],
        [0.1       , 0.27250019],
        [0.2       , 0.60719755],
        [0.1       , 0.32349083],
        [0.1       , 0.204578  ],
        [0.2       , 0.24298202],
        [0.1       , 0.2308373 ],
        [0.1       , 0.20845397],
        [0.1       , 0.2643133 ],
        [0.4       , 0.70882925],
        [0.1       , 0.12955605],
        [0.5       , 0.92478218],
        [0.1       , 0.19306883],
        [0.1       , 0.4975812 ],
        [0.7       , 0.60

In [35]:
b = getBatches(50,'val')
print(b[0])
for e in b:
    print("batch")
    print(e[0])
    print("labels")
    print(e[1])

[[44.          0.1         0.50841713  0.08553381  0.53517689]
 [46.          0.2         0.60729429  0.12884218  0.4772629 ]
 [ 1.          0.2         0.57496762  0.06676959  0.49503099]
 [26.          0.8         0.4423435   0.08985247  0.53127569]
 [10.          0.1         0.19749402  0.10574879  0.48213055]
 [37.          0.2         0.72045199  0.10598889  0.50047276]
 [33.          0.1         0.17952963  0.07988382  0.51093069]
 [28.          0.5         0.77879305  0.08431752  0.50113092]
 [45.          0.3         0.3825154   0.11973342  0.51477755]
 [10.          0.2         0.41174777  0.11872932  0.51153308]
 [19.          0.6         0.91745255  0.09754013  0.50591582]
 [44.          1.4         1.01408995  0.06643108  0.51147607]
 [28.          0.4         0.72354317  0.07215503  0.51536523]
 [ 4.          0.3         1.00448368  0.11172501  0.49728558]
 [39.          4.9         0.92885838  0.10237936  0.53497897]
 [19.          0.3         0.44468154  0.08418972  0.49

In [40]:
tf.reset_default_graph()
%matplotlib notebook
learning_rate = 0.05
batch_size = 100
training_epochs = 100
display_step = 1
complexInput=False
n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 256 # 2nd layer number of neurons
n_input = 5 
n_classes = 2 
step = 1

# tf Graph input
X = tf.placeholder("float32", [None,n_input])
Y = tf.placeholder("float32",[None,n_classes])

keep_probability = tf.placeholder("float")

weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1],stddev=1/n_input)),#**(-1/2))),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2],stddev=1/n_hidden_1)),#**(-1/2))),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes],stddev=1/n_hidden_2))#**(-1/2)))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'bout': tf.Variable(tf.random_normal([n_classes]))
}

def multilayer_perceptron(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.nn.dropout(tf.nn.tanh(tf.add(tf.matmul(x, weights['h1']), biases['b1'])),keep_probability)
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.nn.dropout(tf.nn.tanh(tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])),keep_probability)
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['bout']
    return out_layer

outP = multilayer_perceptron(X)

loss_op = tf.reduce_mean(tf.losses.mean_squared_error(Y,outP))
optimizer = tf.train.AdagradOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

tf.summary.scalar("loss", loss_op)
tf.summary.scalar("sample output",(outP[0][0])*10)
tf.summary.scalar("sample target output",(Y[0][1])*100)

merged_summaries = tf.summary.merge_all()

#train_writer = tf.summary.FileWriter("./summaries/trainFFNAreaRisk", tf.get_default_graph())
#validation_writer = tf.summary.FileWriter("./summaries/validationFFNAreaRisk")

INFO:tensorflow:Summary name sample output is illegal; using sample_output instead.
INFO:tensorflow:Summary name sample target output is illegal; using sample_target_output instead.


In [81]:
with tf.Session() as sess:
    init = tf.initialize_all_variables()
    sess.run(init)
    saver = tf.train.Saver()
    kp=1.0
    samplePred = []
    sample = []
    allLoss = []
    for n in range(training_epochs):
        losses = []
        for b in getBatches(batch_size,"train"):
            _,loss,output,_summaries = sess.run([train_op,loss_op,outP,merged_summaries], feed_dict={X: b[0],Y: b[1],keep_probability:kp})  
            lastout = output[1]
            losses = np.append(losses,loss)
            step = step + 1
            #train_writer.add_summary(_summaries, step)
            if (step%1000==0):
                saver.save(sess,"./log/FFNCountyRisk.ckpt")
        #b = getBatches(batch_size,'val')
        #_,Valloss,output,_Valsummaries = sess.run([train_op,loss_op,outP,merged_summaries], feed_dict={X: np.reshape(b[0],(batch_size,5)),Y: np.reshape(b[1],(batch_size,2)),keep_probability:kp})  
        #print("validation Loss: ",Valloss)
        #validation_writer.add_summary(_Valsummaries, step)
        samplePred = np.append(samplePred,lastout[1])
        sample = np.append(sample,b[1][1][1])
        allLoss = np.append(allLoss,np.mean(losses))
        print("epoch: ",n,' - Step: ',step)
        print("pred: ",lastout)
        print("true: ",b[1][1])
        print("loss: ",np.mean(losses))
        #print(losses[0])

epoch:  0  - Step:  110220
pred:  [0.4774189 0.7076149]
true:  [0.3        0.61751538]
loss:  0.21440162701241935
epoch:  1  - Step:  111978
pred:  [0.47104263 0.6899407 ]
true:  [0.3        0.61751538]
loss:  0.14532391411020285
epoch:  2  - Step:  113736
pred:  [0.45230767 0.6816796 ]
true:  [0.3        0.61751538]
loss:  0.14076073122220337
epoch:  3  - Step:  115494
pred:  [0.43947852 0.6772778 ]
true:  [0.3        0.61751538]
loss:  0.13884060591166844
epoch:  4  - Step:  117252
pred:  [0.42990458 0.67373323]
true:  [0.3        0.61751538]
loss:  0.1377136020405182
epoch:  5  - Step:  119010
pred:  [0.42203957 0.6711713 ]
true:  [0.3        0.61751538]
loss:  0.13694285622462038
epoch:  6  - Step:  120768
pred:  [0.41613996 0.66918546]
true:  [0.3        0.61751538]
loss:  0.13638190327549143


KeyboardInterrupt: 

# Make New Predictions:

In [105]:
with tf.Session() as sess:
    init = tf.initialize_all_variables()
    sess.run(init)
    saver = tf.train.Saver()
    kp=1.0
    saver.restore(sess, "C:/Users/Vivi/Desktop/hack4health_RKI-data-scripts/DieInfluenza/log/FFNCountyRisk.ckpt")
    print("model loaded")
    b,s = getLastWeek(52,2016)
    batch_size = s
    _,loss,output= sess.run([train_op,loss_op,outP], feed_dict={X: b[0],Y: b[1],keep_probability:kp}) 
    print(output)



INFO:tensorflow:Restoring parameters from C:/Users/Vivi/Desktop/hack4health_RKI-data-scripts/DieInfluenza/log/FFNCountyRisk.ckpt
model loaded
[[ 0.16190343  0.26939905]
 [ 0.18071105  0.26028764]
 [ 0.20027275  0.24995267]
 [ 0.18109243  0.308092  ]
 [ 0.18318002  0.2569555 ]
 [ 0.2187732   0.2373302 ]
 [ 0.28070986  0.24652326]
 [ 0.21026774  0.24160337]
 [ 0.21880625  0.23871577]
 [ 0.20166217  0.24799705]
 [ 0.29922497  0.34119666]
 [ 0.24498765  0.22526133]
 [ 0.31945425  0.3871367 ]
 [ 0.22266863  0.23556721]
 [ 0.13760798  0.28424716]
 [ 0.5632488   0.34764683]
 [ 0.29372388  0.28559375]
 [ 0.27556258  0.3567878 ]
 [ 0.34379864  0.25657642]
 [ 0.22026695  0.33400238]
 [ 0.23675923  0.22752583]
 [ 0.13796966  0.33786833]
 [ 0.22810633  0.23099148]
 [ 0.22063704  0.4005685 ]
 [ 0.3753649   0.2893504 ]
 [ 0.22229017  0.28102005]
 [ 0.24280046  0.22244704]
 [ 0.29582512  0.4091305 ]
 [ 0.21961214  0.23897946]
 [ 0.22209965  0.39632082]
 [ 0.3287549   0.21766233]
 [ 0.44779825  0.3001

In [71]:
mses = []
for b in getBatches(batch_size,'train'):
    A = [b[0][0][1],b[0][0][2]]
    B = b[1][0]
    mse = ((A-B)**2).mean()
    mses.append(mse)